In [1]:
import numpy as np
import cvxpy as cp
from scipy import stats

Classic Campi formulation
$$
\begin{aligned}
\texttt{min}_{x \in \mathbb{R}} & f(x)  \\
\texttt{s.t.} & \gamma_i x \leq \beta_i - {\alpha_i} \cdot \chi_j^d, \chi_j^d \sim N(0, I_n),  j= 1,\dots, N \\
\end{aligned}
$$


In [8]:
def normalize_ineqs(A, b):
    norms = np.linalg.norm(A, axis=1)
    return A / norms.reshape(A.shape[0], 1), b / norms
def scenario_approximation_constraints(gamma, beta, alpha, samples):
    Gamma = np.concatenate([gamma for i in range(samples.shape[0])], axis=0)
    print("Gamma : {}".format(len(alpha)))
    print("sample : {}".format(len(samples[0])))
    print("beta : {}".format(len(beta)))
    Beta = np.concatenate([beta - alpha.T @ samples[i] for i in range(samples.shape[0])], axis=0)
    return Gamma, Beta

In [12]:
# CVXPY part formulation

N = 1
eta = 0.01
d = 2
gamma = 2
gamma_ = np.array([[gamma], [-gamma]])
beta = 8
beta_ = np.array([beta, beta])
gamma, beta = normalize_ineqs(gamma_, beta_)
alpha = gamma
#c = np.random.randint(d)
c = [-1]*N


x = cp.Variable(N)
func = c@x
chi = np.random.multivariate_normal(np.zeros(d), np.eye(d), size=N) 
SA_Gamma, SA_Beta = scenario_approximation_constraints(gamma, beta, alpha, chi)
print(SA_Gamma[0], SA_Beta[0])

constraints = []
for i in range(len(SA_Gamma) - 1):
    constraints += [SA_Gamma[i] @ x <= SA_Beta[i]]

prob = cp.Problem(cp.Minimize(func), constraints)
prob.solve()

print("\nObjective function value is", prob.value)
print("A solution x is")
print(x.value)

Gamma : 2
sample : 2
beta : 2
[1.] 4.10485607021374

Objective function value is -4.104856052262742
A solution x is
[4.10485605]


Now scenario approximation with slack constraints

$$
\begin{aligned}
\texttt{min}_{x \in \mathbb{R}} & f(x)  \\
\texttt{s.t.} & \gamma_i x \leq \beta_i - {\alpha_i} \cdot \chi_j^d, \\
&\chi_j^d \sim N(0, I_n),  j= 1,\dots, N \\
& x \in \mathcal{O} = \left\{ x: \forall ~ i ~ \gamma_i x \leq \beta_i + \Phi^{-1}(\eta) \right\}
\end{aligned}
$$

In [15]:
# CVXPY part formulation

N = 1
eta = 0.01
d = 2
gamma = 2
gamma_ = np.array([[gamma], [-gamma]])
beta = 8
beta_ = np.array([beta, beta])
gamma, beta = normalize_ineqs(gamma_, beta_)
alpha = gamma
#c = np.random.randint(d)
c = [-1]*N


x = cp.Variable(N)
func = c@x
chi = np.random.multivariate_normal(np.zeros(d), np.eye(d), size=N)
Phi_inv = stats.norm.ppf(eta)
beta_O = beta + Phi_inv
SA_Gamma, SA_Beta = scenario_approximation_constraints(gamma, beta, alpha, chi)
SCSA_Gamma = np.concatenate([SA_Gamma, gamma], axis=0)
SCSA_Beta = np.concatenate([SA_Beta, beta_O], axis=0)
#print(SA_Gamma, SA_Beta)

constraints = []
for i in range(len(SA_Gamma) - 1):
    constraints += [SA_Gamma[i] @ x <= SA_Beta[i]]

prob = cp.Problem(cp.Minimize(func), constraints)
prob.solve()

print("\nObjective function value is", prob.value)
print("A solution x is")
print(x.value)

Gamma : 2
sample : 2
beta : 2

Objective function value is -5.4175503807897964
A solution x is
[5.41755038]
